In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/creditcard.csv')

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.shape

(284807, 31)

In [6]:
bdf = BaseDatasetTransform(df,target='Class')

In [7]:
df,categorical_features = bdf.fit_transform()


В колонке Class нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index([], dtype='object')

Числовые колонки:
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [8]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
Class
0    284315
1       492
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n


Разделение датасета выполнено успешно


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

def preprocess_and_resample(X_train, y_train, X_test, categorical_cols, random_state=42):

    X_train_copy = X_train.copy()
    X_test_copy = X_test.copy()
    

    label_encoders = {col: LabelEncoder() for col in categorical_cols if col in X_train_copy.columns}
    
    for col, le in label_encoders.items():
        X_train_copy[col] = le.fit_transform(X_train_copy[col])
        X_test_copy[col] = le.transform(X_test_copy[col])

    smote = SMOTE(random_state=random_state)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_copy, y_train)

    X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X_train_copy.columns)
    X_test_processed_df = X_test_copy.copy()

    print("Shape of resampled X_train:", X_train_resampled_df.shape)
    print("Shape of processed X_test:", X_test_processed_df.shape)

    return X_train_resampled_df, y_train_resampled, X_test_processed_df




In [10]:
from sklearn.utils.class_weight import compute_class_weight 
#is_unbalance=True

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

class_counts = y_train.value_counts()


positive_class_count = class_counts[1]  
negative_class_count = class_counts[0]  

scale_pos_weight = negative_class_count / positive_class_count

lgbm_model = lgb.LGBMClassifier(verbose=-1,class_weight=class_weights_dict,scale_pos_weight=scale_pos_weight)


columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [11]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=[],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(class_weight={0: 0.5008642347502763,
                                                                               1: 289.7732558139535},
                                                                 metric='binary_error',
                                                                 objective='binary',
                                                                 scale_pos_weight=578.546511627907,
                                                                 verbose=-1)))

In [12]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.997753
1,precision,0.711653
2,recall,0.911189
3,f1,0.779253
